In [1]:
# !pip3 install seaborn
# Import necessary libraries and helper functions
import pandas as pd
import os
from helper_functions import make_can_df, add_time_diff_per_aid_col, add_actual_attack_col, add_kde_val_col, add_gauss_val_col, get_results_binning, unpickle
import json
import tqdm
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt


# Loading and saving normal traffic


In [2]:
def load_data(directory, exclude=[], file_condition=lambda file_name: True):
    print("Loading data from directory: " + directory)
    df_aggregation = []

    for file_name in os.listdir(directory):
        if file_condition(file_name) and not any(excl in file_name for excl in exclude):
            print("Loading file: " + file_name)
            df = make_can_df(os.path.join(directory, file_name))
            df = add_time_diff_per_aid_col(df)
            df_aggregation.append(df)
            
    return df_aggregation



def load_and_save_training_data(directory):
    if os.path.exists(directory + 'training_data.csv'):
        print("Training data already exists. Loading training data from: " + directory + 'training_data.csv')
        return pd.read_csv(directory + 'training_data.csv')
        
    df_aggregation = load_data(directory, file_condition=lambda file_name: "dyno" in file_name)
    # Concatenate all training datasets on the dyno
    df_training = pd.concat(df_aggregation)
    training_data = df_training[["time", "aid", "time_diffs"]]
    save_file = 'training_data.csv'
    print("Saving training data to: " + save_file)
    training_data.to_csv(directory + save_file)
    return training_data

In [3]:
training_data = load_and_save_training_data('/home/jbc0071/Documents/can-time-based-ids-benchmark/data/ambient/')
display(training_data)


Training data already exists. Loading training data from: /home/jbc0071/Documents/can-time-based-ids-benchmark/data/ambient/training_data.csv


,Unnamed: 0,time,aid,time_diffs
0,4299,1.905791,6,0.999828
1,6680,2.905764,6,0.999973
2,9071,3.906710,6,1.000946
3,11459,4.906623,6,0.999913
4,13851,5.906553,6,0.999930
...,...,...,...,...
13300754,1423478,657.391235,1788,0.100133
13300755,1423627,657.491138,1788,0.099903
13300756,1423776,657.591222,1788,0.100084
13300757,1423926,657.691129,1788,0.099907


## "Train" model: calculating stats

In [4]:
from sklearn.covariance import EllipticEnvelope
import scipy.stats

def preprocess(df, aid):
    """
    Preprocesses the data by removing outliers.
    """
    time_diffs = df[df.aid==aid].time_diffs.values
    print("before: ", len(time_diffs))

    # identify outliers in the dataset
    ee = EllipticEnvelope(contamination=0.0001, support_fraction=0.999) # support_fraction=0.99
    inliers = ee.fit_predict(time_diffs.reshape(-1, 1))

    # select all rows that are not outliers
    mask = inliers != -1
    outliers = sum(mask == False)
    print("outliers: ", outliers, 100*outliers/len(time_diffs))

    time_diffs = time_diffs[mask]
    # summarize the shape of the updated training dataset
    print("after: ", len(time_diffs))

    return time_diffs


def calculate_statistics(time_diffs):
    """
    Returns a dictionary including the mean of its time_diffs, standard deviation of its time_diffs
    and KDE of its time_diffs
    """
    aid_dict = {'mu': time_diffs.mean(), 'std': time_diffs.std(), 'kde': scipy.stats.gaussian_kde(time_diffs), 'gauss': scipy.stats.norm(loc = time_diffs.mean(), scale = time_diffs.std())}
    aid_dict["y_thresholds_kde"] = {}
    aid_dict["y_thresholds_gauss"] = {}
    return aid_dict


def calculate_statistics_for_each_aid(data):
    # Get a list of unique aids in the data
    unique_aids = data['aid'].unique()
    # Preprocess the data and calculate statistics for each unique aid
    stats = {aid: calculate_statistics(preprocess(data, aid)) for aid in unique_aids}
    # data = [add_kde_val_col(data[i], stats) for i in range(len(data))]
    # data = [add_gauss_val_col(data[i], stats) for i in range(len(data))]
    
    return stats




In [5]:
training_data_stats = calculate_statistics_for_each_aid(training_data)
display(training_data_stats)

before:  6340
outliers:  1 0.015772870662460567
after:  6339
before:  580454
outliers:  59 0.010164457476389171
after:  580395
before:  7656
outliers:  1 0.013061650992685475
after:  7655
before:  570718
outliers:  58 0.010162637239407203
after:  570660
before:  11572
outliers:  2 0.017283097131005877
after:  11570
before:  64264
outliers:  7 0.010892568156355035
after:  64257
before:  57860
outliers:  6 0.010369858278603527
after:  57854
before:  6441
outliers:  1 0.01552553951249806
after:  6440
before:  11569
outliers:  2 0.017287578874578616
after:  11567
before:  578687
outliers:  58 0.010022689294903808
after:  578629
before:  160636
outliers:  17 0.010582932841953236
after:  160619
before:  285343
outliers:  29 0.01016320708761035
after:  285314
before:  6419
outliers:  1 0.015578750584203147
after:  6418
before:  570727
outliers:  58 0.010162476981113562
after:  570669
before:  57864
outliers:  6 0.010369141435089175
after:  57858
before:  12850
outliers:  2 0.01556420233463035

{6: {'mu': 1.0155329309177052,
  'std': 0.4947486037052496,
  'kde': <scipy.stats._kde.gaussian_kde at 0x7f94f96d1690>,
  'gauss': <scipy.stats._distn_infrastructure.rv_continuous_frozen at 0x7f942d073b50>,
  'y_thresholds_kde': {},
  'y_thresholds_gauss': {}},
 14: {'mu': 0.010000053884418625,
  'std': 0.00044910612086295176,
  'kde': <scipy.stats._kde.gaussian_kde at 0x7f942d250810>,
  'gauss': <scipy.stats._distn_infrastructure.rv_continuous_frozen at 0x7f942d272a10>,
  'y_thresholds_kde': {},
  'y_thresholds_gauss': {}},
 37: {'mu': 0.8426860569506046,
  'std': 0.5836586794325014,
  'kde': <scipy.stats._kde.gaussian_kde at 0x7f942d0762d0>,
  'gauss': <scipy.stats._distn_infrastructure.rv_continuous_frozen at 0x7f942d088c50>,
  'y_thresholds_kde': {},
  'y_thresholds_gauss': {}},
 51: {'mu': 0.01000002184149984,
  'std': 0.0014177402701057902,
  'kde': <scipy.stats._kde.gaussian_kde at 0x7f942d083d90>,
  'gauss': <scipy.stats._distn_infrastructure.rv_continuous_frozen at 0x7f942d088

## Loading and Annotating Attack Data

In [6]:
def annotate_attack_data(attack_data, injection_intervals):
    """
    Annotates the attack data based on the injection intervals.
    """
    for index, row in injection_intervals.iterrows():
        aid = row['aid']
        payload = row['payload']
        intervals = [(row['start_time'], row['end_time'])]
        attack_data = add_actual_attack_col(attack_data, intervals, aid, payload)
    return attack_data

def load_and_annotate_attack_data(directory, metadata_file):
    # Load the attack data
    df_aggregation = load_data(directory, exclude=['masquerade', 'accelerator', 'metadata', metadata_file])

    # Load the injection intervals from the metadata file
    with open(os.path.join(directory, metadata_file), "r") as read_file:
        attack_dict = json.load(read_file)

    attack_metadata = []
    count = 0  # Initialize count here
    for file_name in os.listdir(directory):
        file_base = file_name[:-4]
        if file_base not in attack_dict:
            continue
        if "masquerade" in file_name or "accelerator" in file_name:
            continue

        metadata = attack_dict[file_base]
        if metadata["injection_id"] != "XXX":
            injection_id = int(metadata["injection_id"], 16)
        else:
            injection_id = "XXX"

        # From metadata file
        attack_metadata.append([tuple(metadata["injection_interval"])])

        # Add column to each attack dataframe to indicate attack (True) or non-attack (False) for each signal
        df_aggregation[count] = add_actual_attack_col(df_aggregation[count], attack_metadata[count], injection_id, metadata["injection_data_str"])

        count += 1  # Increment count here, inside the loop where you add items to your lists

    return df_aggregation


In [7]:
attack_data = load_and_annotate_attack_data('../data/attacks', 'capture_metadata.json')

Loading data from directory: ../data/attacks
Loading file: fuzzing_attack_3.log
Loading file: fuzzing_attack_1.log
Loading file: reverse_light_on_attack_1.log
Loading file: correlated_signal_attack_1.log
Loading file: reverse_light_on_attack_3.log
Loading file: max_speedometer_attack_3.log
Loading file: max_speedometer_attack_1.log
Loading file: correlated_signal_attack_2.log
Loading file: reverse_light_off_attack_3.log
Loading file: fuzzing_attack_2.log
Loading file: correlated_signal_attack_3.log
Loading file: reverse_light_on_attack_2.log
Loading file: reverse_light_off_attack_2.log
Loading file: reverse_light_off_attack_1.log
Loading file: max_engine_coolant_temp_attack.log
Loading file: max_speedometer_attack_2.log


In [8]:
print("Sample attack data from 'reverse_light_on_attack_1.log':")
display(attack_data[2].head())

print("\nHow many have an actual attack:")
print(len(attack_data[2][attack_data[2].actual_attack == True]))

print("\nHow many do not have an actual attack:")
print(len(attack_data[2][attack_data[2].actual_attack == False]))


Sample attack data from 'reverse_light_on_attack_1.log':


,time,aid,data,time_diffs,actual_attack
3958,1.653874,6,0800006400000000,1.000099,False
6348,2.653744,6,0800006400000000,0.999870,False
8739,3.653922,6,0800006400000000,1.000178,False
11131,4.653925,6,0800006400000000,1.000003,False
13522,5.653920,6,0800006400000000,0.999995,False



How many have an actual attack:
3982

How many do not have an actual attack:
120867


## Supporting functions for detecting attacks on data already collected

In [9]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix

def get_results_binning(attack_list, D, n=6):
    """
    Simplified binning detection method that returns the results directly
    """
    results_binning = {}

    for i, attack in enumerate(attack_list):
        confusion_matrix_ = alert_by_bin(attack, D, n)
        precision = confusion_matrix_[1,1] / (confusion_matrix_[1,1] + confusion_matrix_[0,1])
        recall = confusion_matrix_[1,1] / (confusion_matrix_[1,1] + confusion_matrix_[1,0])
        false_positive = confusion_matrix_[0,1] / (confusion_matrix_[0,1] + confusion_matrix_[0,0])

        results_binning[i+1] = {
            'cm': confusion_matrix_,
            'prec': precision,
            'recall': recall,
            'false_pos': false_positive
        }

    return results_binning

def alert_by_bin(df, D, n=6):
    """
    Checks for time windows of length mu*4 (where mu is average time_diff for aid) with 6 or more signals
    """
    cm = np.array([[0,0], [0,0]])
    
    for aid in df.aid.unique():
        df_test = df[df.aid == aid]
        df_test['predicted_attack'] = df_test.time_diffs.rolling(n).sum() <= D[aid]['mu']*4

        cm += confusion_matrix(df_test['actual_attack'], df_test['predicted_attack'], labels = [0,1])
    
    return cm

In [ ]:

    
# def detect_anomalies(models, attack_data, detection_method):
#     if detection_method == 'Binning':
#         return detect_anomalies_binning(models, attack_data)
#     # elif detection_method == 'Gaussian':
#     #     return detect_anomalies_gaussian(models, attack_data)
#     else:
#         raise ValueError(f"Unknown detection method: {detection_method}")


## Live Binning Attack Detection
This is implemented through a binning strategy for anomaly detection, where each bin corresponds \
to a time window of length mu*4. If a bin contains n or more messages, it's considered anomalous \
and marked as a potential attack.

In [10]:
'''
    -1: not enough frames
    0: not attack
    1: attack detected
'''
class BinningAttackDetector:
    """
    This class implements a binning strategy for anomaly detection, where each bin corresponds 
    to a time window of length mu*4. If a bin contains n or more messages, it's considered 
    anomalous and marked as a potential attack.
    """
    def __init__(self, d, n=6):
        self.d = d
        self.n = n
        self.frames = {}

    def process_frame(self, frame):
        aid = frame['aid']
        time_diff = frame['time_diffs']

        # Initialize memory for this aid if it doesn't exist
        if aid not in self.frames:
            self.frames[aid] = []

        # Add this frame to memory
        self.frames[aid].append(time_diff)

        # If we don't have enough frames yet, return 'not enough frames'
        if len(self.frames[aid]) < self.n:
            return -1

        # If we have more than n frames, discard the oldest one
        if len(self.frames[aid]) > self.n:
            self.frames[aid].pop(0)

        # Check if the sum of time_diffs for the last n frames is less than or equal to mu*4
        if sum(self.frames[aid]) <= self.d[aid]['mu']*4:
            return 1
        else:
            return 0

detector = BinningAttackDetector(training_data_stats)

from sklearn.metrics import confusion_matrix

def calculate_cm(df, predictions):
    cm = confusion_matrix(df['actual_attack'], predictions, labels=[False, True])
    return cm


## Calculation of metrics and total accuracy

In [11]:
# modified from get_results_binning in helper_functions.py
def get_results_binning_updated(attack_list, D, n=6):
    # Initialize results dictionary
    results_binning = {}
    for i in range(len(attack_list)):
        results_binning[i+1] = {'cm': [0], 'recall': 0, 'prec': 0, 'false_pos': 0}
    results_binning['total'] = {'cm': [0], 'recall': 0, 'prec': 0, 'false_pos': 0}

    for i in range(len(attack_list)):
        # Process each frame in the attack
        predictions = []
        for _, frame in attack_list[i].iterrows():
            result = detector.process_frame(frame.to_dict())
            predictions.append(result == 'attack')

        # Calculate the confusion matrix
        results_binning[i+1]['cm'] = calculate_cm(attack_list[i], predictions)

        # Calculate the other metrics here...
        denominator = (results_binning[i+1]['cm'][1, 1]+results_binning[i+1]['cm'][0, 1])
        if denominator != 0:
            results_binning[i+1]['prec'] = results_binning[i+1]['cm'][1, 1] / denominator
        else:
            results_binning[i+1]['prec'] = 0  # or np.nan, or however you want to handle this case

        results_binning[i+1]['recall'] = results_binning[i+1]['cm'][1, 1] / \
            (results_binning[i+1]['cm'][1, 1]+results_binning[i+1]['cm'][1, 0])
        results_binning[i+1]['false_pos'] = results_binning[i+1]['cm'][0, 1] / \
            (results_binning[i+1]['cm'][0, 1]+results_binning[i+1]['cm'][0, 0])
        results_binning['total']['cm'] += results_binning[i+1]['cm']

    # Calculate the total metrics here...
    results_binning['total']['prec'] = results_binning['total']['cm'][1, 1] / \
        (results_binning['total']['cm'][1, 1] +
         results_binning['total']['cm'][0, 1])
    results_binning['total']['recall'] = results_binning['total']['cm'][1, 1] / \
        (results_binning['total']['cm'][1, 1] +
         results_binning['total']['cm'][1, 0])
    results_binning['total']['f1'] = 2*((results_binning['total']['prec']*results_binning['total']['recall'])/(
        results_binning['total']['prec']+results_binning['total']['recall']))
    results_binning['total']['false_pos'] = results_binning['total']['cm'][0, 1] / \
        (results_binning['total']['cm'][0, 1] +
         results_binning['total']['cm'][0, 0])


    return results_binning



In [12]:
result_binning = get_results_binning(attack_data, training_data_stats)

/tmp/ipykernel_308142/39001723.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['predicted_attack'] = df_test.time_diffs.rolling(n).sum() <= D[aid]['mu']*4
/tmp/ipykernel_308142/39001723.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['predicted_attack'] = df_test.time_diffs.rolling(n).sum() <= D[aid]['mu']*4
/tmp/ipykernel_308142/39001723.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [13]:
result_binning_new = get_results_binning_updated(attack_data, training_data_stats)

In [31]:
detector = BinningAttackDetector(training_data_stats)

display(type(attack_data[4]))
for frame in attack_data[4].to_dict():
    display(frame, type(frame))
    if detector.process_frame(frame) != 'attack':
        print("attack")


pandas.core.frame.DataFrame

'time'

str

TypeError: string indices must be integers, not 'str'

In [14]:
display(result_binning)
display(result_binning_new)

{1: {'cm': array([[12182,     0],
         [    3,     0]]),
  'prec': nan,
  'recall': 0.0,
  'false_pos': 0.0},
 2: {'cm': array([[45457,    92],
         [   36,     0]]),
  'prec': 0.0,
  'recall': 0.0,
  'false_pos': 0.0020198028496783683},
 3: {'cm': array([[120692,    175],
         [     2,   3980]]),
  'prec': 0.9578820697954272,
  'recall': 0.9994977398292315,
  'false_pos': 0.0014478724548470632},
 4: {'cm': array([[71954,     5],
         [    1,  4171]]),
  'prec': 0.9988026819923371,
  'recall': 0.9997603068072867,
  'false_pos': 6.948401172890118e-05},
 5: {'cm': array([[141427,    119],
         [     3,   4699]]),
  'prec': 0.9753009547530096,
  'recall': 0.9993619736282433,
  'false_pos': 0.0008407160922950843},
 6: {'cm': array([[188114,     14],
         [     2,  12212]]),
  'prec': 0.9988548993947325,
  'recall': 0.9998362534796136,
  'false_pos': 7.441741792821908e-05},
 7: {'cm': array([[194489,    624],
         [     2,   4886]]),
  'prec': 0.8867513611615245,

{1: {'cm': array([[12182,     0],
         [    3,     0]]),
  'recall': 0.0,
  'prec': 0,
  'false_pos': 0.0},
 2: {'cm': array([[45457,    92],
         [   36,     0]]),
  'recall': 0.0,
  'prec': 0.0,
  'false_pos': 0.0020198028496783683},
 3: {'cm': array([[120687,    180],
         [     2,   3980]]),
  'recall': 0.9994977398292315,
  'prec': 0.9567307692307693,
  'false_pos': 0.001489240239271265},
 4: {'cm': array([[71953,     6],
         [    1,  4171]]),
  'recall': 0.9997603068072867,
  'prec': 0.998563562365334,
  'false_pos': 8.338081407468142e-05},
 5: {'cm': array([[141424,    122],
         [     3,   4699]]),
  'recall': 0.9993619736282433,
  'prec': 0.974694046878241,
  'false_pos': 0.0008619106156302545},
 6: {'cm': array([[188112,     16],
         [     2,  12212]]),
  'recall': 0.9998362534796136,
  'prec': 0.9986915276414786,
  'false_pos': 8.504847763225038e-05},
 7: {'cm': array([[194484,    629],
         [     2,   4886]]),
  'recall': 0.9995908346972177,
  

# Visualize each file for result_binning_new

# KDE Method:

In [ ]:

from helper_functions import y_threshold_kde

display(attack_data[1].aid)

# from original script: Not sure what it is exactly yet.
pvals_kde = sorted(list(np.arange(0.001, 0.01, 0.001)) + list(np.arange(0, 0.1, 0.01)))

# Fill each 'y_thresholds_kde' dictionary with p values and their corresponding y_thresholds
# Once this has been done, results are pickled and don't need to be re-run.
# Will comment this out to avoid overwriting
#cc = 0
for aid in tqdm(attack_data.aid.unique()):
    #cc += 1
    #print (f'aid {cc} of 128')
    for p in pvals_kde:
        y_threshold_kde(training_dict, aid, p)

In [ ]:
""" Original:
    Marks as attack when last three time_diffs had had p-value less than the p-value threshold for kde
    Calculates confusion matrix, precision, recall, false positive rate and saves to dictionary
    """
def get_results_kde_updated(pvals, attack_list, d):
    pvals_kde = sorted(list(np.arange(0.001, 0.01, 0.001)) + list(np.arange(0, 0.1, 0.01)))

    results_kde_final = {}
    for p in pvals_kde:
        results_kde_final[p] = {'cm': [0], 'recall': 0, 'prec': 0, 'false_pos': 0}

    for p in pvals:
        details = np.array([[0, 0], [0, 0]])
        for i in range(len(attack_list)):
            # Process each frame in the attack
            predictions = []
            for _, frame in attack_list[i].iterrows():
                result = detector.process_frame(frame.to_dict())
                predictions.append(result == 'attack')

            # Calculate the confusion matrix
            details += calculate_cm(attack_list[i], predictions)

        results_kde_final[p]['cm'] = details

        # Calculate the other metrics here...
        denominator = (results_kde_final[p]['cm'][1, 1]+results_kde_final[p]['cm'][0, 1])
        if denominator != 0:
            results_kde_final[p]['prec'] = results_kde_final[p]['cm'][1, 1] / denominator
        else:
            results_kde_final[p]['prec'] = 0  # or np.nan, or however you want to handle this case

        results_kde_final[p]['recall'] = results_kde_final[p]['cm'][1, 1] / \
            (results_kde_final[p]['cm'][1, 1]+results_kde_final[p]['cm'][1, 0])
        results_kde_final[p]['false_pos'] = results_kde_final[p]['cm'][0, 1] / \
            (results_kde_final[p]['cm'][0, 1]+results_kde_final[p]['cm'][0, 0])

        results_kde_final[p]['f1'] = 2 * ((results_kde_final[p]['prec'] * results_kde_final[p]['recall']) / 
                                          (results_kde_final[p]['prec'] + results_kde_final[p]['recall']))

    return results_kde_final
    # picklify(results_kde_final, os.path.dirname(os.getcwd()) + "/results_kde_final.pkl")



In [ ]:
results_kde_final = get_results_kde_updated(pvals_kde, attack_data, 

## Unfinished below

In [ ]:

# bin_results = []
# for i in range(len(attack_data)):
#     result_binning_new = get_results_binning_updated([attack_data[i]], training_data_stats)
#     bin_results.append(result_binning_new)

In [ ]:
# for result in bin_results:
#     display(result['total'])



In [ ]:
display(attack_data)
display(training_data_stats)

In [ ]:
results_binning_final = unpickle(os.path.dirname(os.getcwd()) + "/results_binning_final.pkl") 


In [ ]:
display(results_binning_final)

In [ ]:
results = detect_and_save_results('results', training_data_stats, attack_data, 'Binning') # Detect anomalies and save the results
# visualize_results(results)